In [1]:
import pandas as pd
import sqlite3 as lite

In [2]:
db = lite.connect('all_info.db')

In [ ]:
# 이런게 있구나 라고 생각하심 됩니다.
# 1) 커서를 만들어서 sql 날린뒤 결과 fetch하는법(굉장히 비효율적)
cur = db.cursor()
cur.execute("select * from Orders;")
cur.fetchall()

In [10]:
# 2) pandas read_sql로 결과 받아보기
df = pd.read_sql("select * from Orders;", db)
df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [13]:
sql = """
    select name
    from sqlite_master
    where
        type = 'table'
    ;
"""

sql = """
    select sql
    from sqlite_master
    where
        type = 'table'
    ;
"""
pd.options.display.max_colwidth = 120
pd.read_sql(sql, db)

,sql
0,"CREATE TABLE Categories(\n ""CategoryID"" TEXT,\n ""CategoryName"" TEXT,\n ""Description"" TEXT\n)"
1,"CREATE TABLE Customers(\n ""CustomerID"" TEXT,\n ""CustomerName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""..."
2,"CREATE TABLE Employees(\n ""EmployeeID"" TEXT,\n ""LastName"" TEXT,\n ""FirstName"" TEXT,\n ""BirthDate"" TEXT,\n ""Phot..."
3,"CREATE TABLE OrderDetails(\n ""OrderDetailID"" TEXT,\n ""OrderID"" TEXT,\n ""ProductID"" TEXT,\n ""Quantity"" TEXT\n)"
4,"CREATE TABLE Orders(\n ""OrderID"" TEXT,\n ""CustomerID"" TEXT,\n ""EmployeeID"" TEXT,\n ""OrderDate"" TEXT,\n ""Shipper..."
5,"CREATE TABLE Products(\n ""ProductID"" TEXT,\n ""ProductName"" TEXT,\n ""SupplierID"" TEXT,\n ""CategoryID"" TEXT,\n ""U..."
6,"CREATE TABLE Shippers(\n ""ShipperID"" TEXT,\n ""ShipperName"" TEXT,\n ""Phone"" TEXT\n)"
7,"CREATE TABLE Suppliers(\n ""SupplierID"" TEXT,\n ""SupplierName"" TEXT,\n ""ContactName"" TEXT,\n ""Address"" TEXT,\n ""..."


In [23]:
import time
start = time.time()

query = """
    select count(*) 
    from OrderDetails
    ;
"""
count = pd.read_sql(query, db)
print(count)

end = time.time()
duration = end - start
print(duration)

   count(*)
0       518
0.0037970542907714844


In [22]:
start = time.time()

count = len(pd.read_sql("select * from OrderDetails;", db))
print(count)

end = time.time()
duration = end - start
print(duration)

518
0.0048940181732177734


In [26]:
query= """
    select *
    from Customers;
"""
customer_df = pd.read_sql(query, db)
customer_df.head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


In [31]:
is_france = customer_df["Country"] == "France"
is_germany = customer_df["Country"] == "Germany"
is_paris = customer_df["City"] == "Paris"

# pandas에서 다중조건 활용하기 => | 또는 &
customer_df[is_germany | is_france].head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
5,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
6,7,Blondel père et fils,Frédérique Citeaux,24 place Kléber,Strasbourg,67000,France
8,9,Bon app',Laurence Lebihans,12 rue des Bouchers,Marseille,13008,France
16,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany


In [36]:
query = """
    select *
    from Customers
    where
        Country = "Germany"
        or
        Country = "France"
    ;
"""

pd.read_sql(query, db).head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
2,7,Blondel père et fils,Frédérique Citeaux,24 place Kléber,Strasbourg,67000,France
3,9,Bon app',Laurence Lebihans,12 rue des Bouchers,Marseille,13008,France
4,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany


In [40]:
products_df = pd.read_sql('select * from Products;', db)
products_df.sort_values('ProductName', ascending=False)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
46,47,Zaanse koeken,22,3,10 - 4 oz boxes,9.5
63,64,Wimmers gute Semmelknödel,12,5,20 bags x 4 pieces,33.25
62,63,Vegie-spread,7,2,15 - 625 g jars,43.9
49,50,Valkoinen suklaa,23,3,12 - 100 g bars,16.25
6,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30
22,23,Tunnbröd,9,5,12 - 250 g pkgs.,9
53,54,Tourtière,25,6,16 pies,7.45
13,14,Tofu,6,7,40 - 100 g pkgs.,23.25
28,29,Thüringer Rostbratwurst,12,6,50 bags x 30 sausgs.,123.79
18,19,Teatime Chocolate Biscuits,8,3,10 boxes x 12 pieces,9.2


In [42]:
query = """
    select * 
    from Products
    ORDER BY ProductName 
    ;
""" # ASC => 오름차순(기본값), DESC => 내림차순
pd.read_sql(query, db)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,17,Alice Mutton,7,6,20 - 1 kg tins,39
1,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10
2,40,Boston Crab Meat,19,8,24 - 4 oz tins,18.4
3,60,Camembert Pierrot,28,4,15 - 300 g rounds,34
4,18,Carnarvon Tigers,7,8,16 kg pkg.,62.5
5,1,Chais,1,1,10 boxes x 20 bags,18
6,2,Chang,1,1,24 - 12 oz bottles,19
7,39,Chartreuse verte,18,1,750 cc per bottle,18
8,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22
9,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35


In [45]:
query ="""
    select *
    from Products
    where
        ProductName LIKE "%An%"
    ;
""" # LIKE "%Ch%" "Fast_ampus"
pd.read_sql(query, db)

,ProductID,ProductName,SupplierID,CategoryID,Unit,Price
0,2,Chang,1,1,24 - 12 oz bottles,19
1,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10
2,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22
3,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35
4,6,Grandma's Boysenberry Spread,3,2,12 - 8 oz jars,25
5,7,Uncle Bob's Organic Dried Pears,3,7,12 - 1 lb pkgs.,30
6,8,Northwoods Cranberry Sauce,3,2,12 - 12 oz jars,40
7,12,Queso Manchego La Pastora,5,4,10 - 500 g pkgs.,38
8,24,Guaraná Fantástica,10,1,12 - 355 ml cans,4.5
9,41,Jack's New England Clam Chowder,19,8,12 - 12 oz cans,9.65


In [46]:
# How to merge with (1) pandas
query= """
    select * 
    from Customers
    ;
"""
customers_df = pd.read_sql(query, db)

query= """
    select * 
    from Orders
    ;
"""
orders_df = pd.read_sql(query, db)

In [49]:
integrated_df = orders_df.merge(customers_df, on="CustomerID")
integrated_df[["OrderID", "CustomerID", "CustomerName", "Address"]].head()

,OrderID,CustomerID,CustomerName,Address
0,10248,90,Wilman Kala,Keskuskatu 45
1,10249,81,Tradição Hipermercados,Av. Inês de Castro 414
2,10292,81,Tradição Hipermercados,Av. Inês de Castro 414
3,10250,34,Hanari Carnes,Rua do Paço 67
4,10253,34,Hanari Carnes,Rua do Paço 67


In [51]:
#(2) Join with sql
# 이렇게 하지 마세요
query = """
    select *
    from Orders, Customers
    ;

"""

pd.read_sql(query, db).head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,10248,90,5,1996-07-04,3,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,10248,90,5,1996-07-04,3,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico
2,10248,90,5,1996-07-04,3,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico
3,10248,90,5,1996-07-04,3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,10248,90,5,1996-07-04,3,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


In [54]:
query = """
    select *
    from Orders, Customers
    where
        Customers.CustomerID = Orders.CustomerID
    ;

"""

query = """
    select O.OrderID, C.CustomerName
    from Orders O, Customers C
    where
        C.CustomerID = O.CustomerID
    ;

"""

pd.read_sql(query, db)

,OrderID,CustomerName
0,10248,Wilman Kala
1,10249,Tradição Hipermercados
2,10250,Hanari Carnes
3,10251,Victuailles en stock
4,10252,Suprêmes délices
5,10253,Hanari Carnes
6,10254,Chop-suey Chinese
7,10255,Richter Supermarkt
8,10256,Wellington Importadora
9,10257,HILARIÓN-Abastos


In [57]:
# 강추 : join

query = """
    select * 
    from Customers C
    JOIN Orders O
    ON C.CustomerID = O.CustomerID
    ;
"""

pd.read_sql(query, db).head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico,10308,2,7,1996-09-18,3
1,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico,10365,3,3,1996-11-27,2
2,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK,10355,4,6,1996-11-15,1
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK,10383,4,8,1996-12-16,3
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden,10278,5,8,1996-08-12,2


In [59]:
query = """
    select *
    from Orders
    ;
"""

order_df = pd.read_sql(query, db)
order_df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,1996-07-04,3
1,10249,81,6,1996-07-05,1
2,10250,34,4,1996-07-08,2
3,10251,84,3,1996-07-08,1
4,10252,76,4,1996-07-09,2


In [71]:
order_groups = order_df.groupby("OrderDate")
len(order_groups.get_group("1996-07-08"))

order_df["OrderDate"].unique()
order_count = pd.DataFrame([{
    "OrderDate": OrderDate,
    "Count": len(order_groups.get_group(OrderDate)),
} for OrderDate in order_df["OrderDate"].unique()])
order_count.head()

,Count,OrderDate
0,1,1996-07-04
1,1,1996-07-05
2,2,1996-07-08
3,1,1996-07-09
4,1,1996-07-10


In [66]:
show_list = [x*x for x in range(1, 10+1)]
show_list

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [73]:
order_df["month"] = order_df["OrderDate"].apply(lambda x: str(x)[:7])
order_df.head()

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID,month
0,10248,90,5,1996-07-04,3,1996-07
1,10249,81,6,1996-07-05,1,1996-07
2,10250,34,4,1996-07-08,2,1996-07
3,10251,84,3,1996-07-08,1,1996-07
4,10252,76,4,1996-07-09,2,1996-07


In [75]:
order_groups = order_df.groupby("month")
order_count = pd.DataFrame([
    {
    "month": month,
    "Count": len(order_groups.get_group(month)),
    } for month in order_df["month"].unique()])
order_count.head()

,Count,month
0,22,1996-07
1,25,1996-08
2,23,1996-09
3,26,1996-10
4,25,1996-11
